# Setup environment

In [ ]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Import modules
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_classif, chi2, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, cross_validate, GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Move to src directory
os.chdir('drive/MyDrive/CATS_project/src')

# Load and transform the data

In [ ]:
# Load the training set
train_X = pd.read_csv(r'../data/Train_call.txt', sep='	')
display(train_X)

,Chromosome,Start,End,Nclone,Array.129,Array.34,Array.67,Array.24,Array.22,Array.36,...,Array.64,Array.89,Array.30,Array.35,Array.93,Array.10,Array.123,Array.100,Array.134,Array.130
0,1,2927,43870,3,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,-1,0
1,1,85022,216735,4,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,-1,0
2,1,370546,372295,4,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,-1,0
3,1,471671,786483,5,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,-1,0
4,1,792533,907406,13,0,0,0,0,0,0,...,0,1,1,0,1,0,0,0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2829,23,153062077,153452633,57,1,1,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
2830,23,153466463,153491568,4,1,1,1,0,1,1,...,2,1,1,1,1,1,1,1,1,1
2831,23,153504394,153933426,55,1,1,1,0,1,1,...,2,1,1,1,1,1,1,1,1,1
2832,23,153938998,153989329,5,1,1,1,0,1,1,...,2,1,1,1,1,1,1,1,1,1


In [ ]:
# Load the training labels
train_y = pd.read_csv(r'../data/Train_clinical.txt', sep='	', index_col=0)
display(train_y)

,Subgroup
Sample,
Array.129,HER2+
Array.34,HR+
Array.67,HR+
Array.24,Triple Neg
Array.22,Triple Neg
...,...
Array.10,HER2+
Array.123,HR+
Array.100,HR+


In [ ]:
# Combine the first three columns to be the feature names
train_X['featureNames'] = 'chr' + train_X['Chromosome'].astype(str) + ':' + train_X['Start'].astype(str) + '-' + train_X['End'].astype(str)

# Drop the irrelevant columns
train_X.drop(columns=['Chromosome', 'Start', 'End', 'Nclone'], inplace=True)

#Convert the feature names to columns and the columns to indexes
train_X = train_X.set_index('featureNames').T

display(train_X)

featureNames,chr1:2927-43870,chr1:85022-216735,chr1:370546-372295,chr1:471671-786483,chr1:792533-907406,chr1:912799-1266212,chr1:1271190-1590570,chr1:1676445-1703748,chr1:1738295-2477597,chr1:2481927-2562342,...,chr23:151067607-152416606,chr23:152422390-152548587,chr23:152552851-152570071,chr23:152576854-152935130,chr23:152994680-153054487,chr23:153062077-153452633,chr23:153466463-153491568,chr23:153504394-153933426,chr23:153938998-153989329,chr23:153997146-154492924
Array.129,0,0,0,0,0,0,0,0,0,0,...,2,2,2,2,0,1,1,1,1,1
Array.34,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.67,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.24,0,0,0,0,0,0,0,-1,0,0,...,0,0,0,0,0,0,0,0,0,0
Array.22,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Array.10,0,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,1
Array.123,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.100,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.134,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
# Add the labels to the train data
train = train_y.merge(train_X, left_index=True, right_index=True)

display(train)

,Subgroup,chr1:2927-43870,chr1:85022-216735,chr1:370546-372295,chr1:471671-786483,chr1:792533-907406,chr1:912799-1266212,chr1:1271190-1590570,chr1:1676445-1703748,chr1:1738295-2477597,...,chr23:151067607-152416606,chr23:152422390-152548587,chr23:152552851-152570071,chr23:152576854-152935130,chr23:152994680-153054487,chr23:153062077-153452633,chr23:153466463-153491568,chr23:153504394-153933426,chr23:153938998-153989329,chr23:153997146-154492924
Array.129,HER2+,0,0,0,0,0,0,0,0,0,...,2,2,2,2,0,1,1,1,1,1
Array.34,HR+,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.67,HR+,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.24,Triple Neg,0,0,0,0,0,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
Array.22,Triple Neg,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Array.10,HER2+,0,0,0,0,0,0,0,0,0,...,1,1,0,1,1,1,1,1,1,1
Array.123,HR+,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.100,HR+,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
Array.134,HR+,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,1,1,1,1,1,1,1,1,1


## Nested cross validation

In [ ]:
#Function to perform nested cross validation. Only a model and parameter grid should be provided
#The parameter prefix should be 'model__'
def nested_cv(model, train_X, train_y, param_grid, num_inner_folds = 3, num_outer_folds = 4, random_state = 1):

  inner_best_params = []
  inner_best_features = []
  outer_validation_scores = []
  outer_train_scores = []
  outer_models = []

  #Define the inner/outer folds
  inner_cv = KFold(n_splits=num_inner_folds, shuffle=True, random_state=random_state)
  outer_cv = KFold(n_splits=num_outer_folds, shuffle=True, random_state=random_state)

  #Perform inner cross validation to obtain best hyper parameter settings
  pipe = Pipeline(steps=[('feature_selection', SelectKBest()), ('model', model())])

  search_inner = GridSearchCV(pipe, param_grid=param_grid, cv=inner_cv)

  #Perform outer cross validation
  for train, validation in outer_cv.split(train_X):
    search_inner.fit(train_X.iloc[train], np.ravel(train_y.iloc[train]))
    inner_best_params.append(search_inner.best_params_)
    inner_best_features.append(search_inner.feature_names_in_)

    #Create the model arguments from the best inner cv params
    args = {}
    for param, value in search_inner.best_params_.items():
      if param.startswith('model'):
        args[param.replace('model__', '')] = value

    outer_model = RandomForestClassifier(**args)
    outer_model.fit(train_X.iloc[train][search_inner.feature_names_in_], np.ravel(train_y.iloc[train]))

    outer_validation_scores.append(
        outer_model.score(
            train_X.iloc[validation][search_inner.feature_names_in_], 
            np.ravel(train_y.iloc[validation])))
    
    outer_train_scores.append(
        outer_model.score(
            train_X.iloc[train][search_inner.feature_names_in_], 
            np.ravel(train_y.iloc[train])))

    outer_models.append(outer_model)

  return outer_models, outer_validation_scores, outer_train_scores, inner_best_params, inner_best_features



param_grid = {
    'feature_selection__k': [20, 40, 60, 80, 100],
    'model__n_estimators': [500, 1000, 1500, 2000],
    'model__max_features': ['auto', 'sqrt']
}

In [ ]:
#Random forest
outer_models, outer_validation_scores, outer_train_scores, inner_best_params, inner_best_features = nested_cv(RandomForestClassifier, train_X, train_y, param_grid)
print(outer_validation_scores)
print(outer_train_scores)

[0.72, 0.64, 0.76, 0.76]
[1.0, 1.0, 1.0, 1.0]


In [ ]:
#XGboost
outer_models, outer_validation_scores, outer_train_scores, inner_best_params, inner_best_features = nested_cv(XGBClassifier, train_X , train_y, param_grid)
print(outer_validation_scores)
print(outer_train_scores)

[0.64, 0.64, 0.68, 0.72]
[1.0, 1.0, 1.0, 1.0]
